#### http://www.kipris.or.kr/khome/main.jsp
#### ("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"머신러닝"+"machine learning")*AD=[20190624~20240624]

#### 로그인 후 환경설정 셋팅 후 검색
#### 환경설정 - 국제출원일자(FD), 우선권주장일자(PRD)
#### 목록 뷰 스타일 - 서지정보 보기

#### boseon169@gmail.com
#### !@1234boseon


In [11]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import openpyxl

class Kipris:
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.m_page_code = ''
        self.df = pd.DataFrame(columns=['status', 'title', 'ap_num', 'application_date', 'applicant', 'representative',
                                        'final_owner', 'priority_date', 'international_filing_date', 'summary'])

    def open(self):
        # 사이트 열기
        url = 'http://www.kipris.or.kr/khome/main.jsp'
        self.driver.get(url)
        # 처음에 열린 page코드
        page_dict = {idx: self.driver.window_handles[idx] for idx in range(len(self.driver.window_handles))}
        # 메인페이지 코드
        self.m_page_code = page_dict[0]

    def close(self):
        # 브라우저 종료
        self.driver.quit()

    def login(self):
        print('\nlogin() start')
        login = self.driver.find_element(By.ID, 'btnLogin')
        login.click()
        # 로그인 팝업창 선택
        self.driver.switch_to.window(self.driver.window_handles[1])
        # 로그인 ID
        login_id = self.driver.find_element(By.ID, 'login_id')
        login_id.send_keys('boseon169@gmail.com')
        # 로그인 PW
        login_pw = self.driver.find_element(By.ID, 'login_pw')
        login_pw.send_keys('!@1234boseon')
        # 로그인 버튼
        btn_login = self.driver.find_element(By.ID, 'btn_login')
        btn_login.click()
        # 5초 기다린 후 팝업 닫기
        time.sleep(5)
        self.pop_close()
        print('login() end')

    def pop_close(self):
        print('\npop_close() start')
        # 팝업이 있는지 10초동안 확인
        wait = WebDriverWait(self.driver, 10)
        wait.until(lambda d: len(self.driver.window_handles) > 1)
        print('페이지 갯수 : ', len(self.driver.window_handles))
        self.driver.switch_to.window(self.m_page_code)
        # 팝업이 있을 경우 팝업 닫기
        if len(self.driver.window_handles) > 1:
            total_site = self.driver.window_handles
            print(f'총 페이지 : {total_site}')
            print(f'메인 페이지 코드 : {self.m_page_code}')
            for page in total_site:
                if page != self.m_page_code:
                    self.driver.switch_to.window(page)
                    self.driver.close()
            self.driver.switch_to.window(self.m_page_code)
            print('if end')
        print('pop_close() end')

    def patent_move(self):
        print('patent_move start')
        # 특허·실용신안 태그 가져오기
        patent_tag = WebDriverWait(self.driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#gnb ul.cbp-hrsub-inner.cbp-hrsub-inner.depth2 li:nth-child(1) a'))
        )
        # 클릭하여 페이지 이동
        patent_tag.click()
        time.sleep(3)
        # 검색창 입력
        input_tag = self.driver.find_element(By.ID, 'queryText')
        input_tag.send_keys('("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"딥러닝"+"deep learning")*AD=[20130624~20240624]')
        input_tag.send_keys(Keys.ENTER)
        time.sleep(3)
        # 리스트 형식 변경 (요약 함께보기)
        btnSimple = self.driver.find_element(By.CSS_SELECTOR, '#btnTextView')
        btnSimple.click()
        time.sleep(3)
        # 페이지 목록 갯수 설정
        select = Select(self.driver.find_element(By.ID, 'opt28'))
        select.select_by_value('90')
        time.sleep(3)
        # 설정된 페이지 목록 갯수로 다시 조회
        select_a = self.driver.find_element(By.CSS_SELECTOR, '#pageSel a')
        select_a.click()
        print('patent_move end')

    def page_num_move(self, current_page, total_pages):
        # 페이지 번호 이동 및 10페이지마다 다음 버튼 클릭
        if current_page % 10 == 0:
            next_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[@class="next"]'))
            )
            next_button.click()
        else:
            page_link = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="divMainArticle"]/form/section/div[2]/span/a[text()="{current_page + 1}"]'))
            )
            page_link.click()
        time.sleep(3)

##                 console.log(i+"번째 특허")
    def data_setting(self):
        print('\ndata_setting start')
        data = self.driver.execute_script("""
            var main_section = document.querySelector('#divMainArticle section.search_section');
            var divs = main_section.querySelectorAll('[id*="divViewSel"]');
            var status_list = [];
            var title_list = [];
            var ap_num_list = [];
            var application_date_list = [];
            var applicant_list = [];
            var representative_list = [];
            var final_owner_list = [];
            var priority_date_list = [];
            var international_filing_date_list = [];
            var summary_list = [];
            for (var i = 0; i < divs.length; i++) {
                var div = divs[i];
                var h1_tag = div.querySelector('h1[class*="stitle"]');
                var statusEl = h1_tag.querySelector('#iconStatus');
                var status = statusEl ? statusEl.textContent : '';
                status_list.push(status);
                var titleEl = h1_tag.querySelector('a:nth-of-type(2)');
                var title = titleEl ? titleEl.textContent : '';
                title_list.push(title);
                var divRealContent = div.querySelector('#divRealContent' + i);
                var ap_num_el = divRealContent.querySelector('li:nth-child(2) span.point01');
                var ap_num = ap_num_el ? ap_num_el.textContent : '';
                ap_num_list.push(ap_num);
                var ap_date_el = divRealContent.querySelector('li:nth-child(3)');
                var ap_date = ap_date_el ? ap_date_el.textContent.replace(/[^\d]/g, '') : '';
                application_date_list.push(ap_date);
                var applicant_el = divRealContent.querySelector('ul.search_info_list li:nth-child(4) a');
                var applicant = applicant_el ? applicant_el.textContent : '';
                applicant_list.push(applicant);
                var representative_el = divRealContent.querySelector('ul.search_info_list li:nth-child(10) a');
                var representative = representative_el ? representative_el.textContent : '';
                representative_list.push(representative);
                var final_owner_el = divRealContent.querySelector('ul.search_info_list li:nth-child(11) a');
                var final_owner = final_owner_el ? final_owner_el.textContent : '';
                final_owner_list.push(final_owner);
                var div_search_detail = divRealContent.querySelector('#div_search_detail_' + ap_num);
                var priority_date_el = div_search_detail.querySelector('ul li:nth-child(1)');
                var priority_date = priority_date_el ? priority_date_el.textContent.replace(/[^\d]/g, '') : '';
                priority_date_list.push(priority_date);
                var international_filing_date_el = div_search_detail.querySelector('ul li:nth-child(2)');
                var international_filing_date = international_filing_date_el ? international_filing_date_el.textContent.replace(/[^\d]/g, '') : '';
                international_filing_date_list.push(international_filing_date);
                var summary_el = div_search_detail.querySelector('div.search_txt');
                var summary = summary_el ? summary_el.textContent.replace(/[\\n\\t]+/g, '') : '';
                summary_list.push(summary);
            }
            return {
                'status': status_list,
                'title': title_list,
                'ap_num': ap_num_list,
                'application_date': application_date_list,
                'applicant': applicant_list,
                'representative': representative_list,
                'final_owner': final_owner_list,
                'priority_date': priority_date_list,
                'international_filing_date': international_filing_date_list,
                'summary': summary_list
            };
        """)
        df = pd.DataFrame({
            'status': data['status'],
            'title': data['title'],
            'ap_num': data['ap_num'],
            'application_date': data['application_date'],
            'applicant': data['applicant'],
            'representative': data['representative'],
            'final_owner': data['final_owner'],
            'priority_date': data['priority_date'],
            'international_filing_date': data['international_filing_date'],
            'summary': data['summary']
        })
        print('data_setting end\n')
        return df

    def scrape_data(self):
        # 페이지 오픈
        self.open()
        # 팝업 닫기
        self.pop_close()
        # 로그인 진행
        self.login()
        # 페이지 이동
        self.patent_move()
        time.sleep(10)
        articles_info = self.driver.find_element(By.CSS_SELECTOR, 'p.articles').text
        total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())  # 총 페이지
        # 처음 1페이지 데이터 저장
        self.df = pd.concat([self.df, self.data_setting()], ignore_index=True)
        #for i in range(1, 15):
        for i in range(1, total_pages):
            current_page = int(self.driver.find_element(By.CSS_SELECTOR, '#divMainArticle div.float_left span.current').text)  # 현재 페이지
            self.page_num_move(current_page, total_pages)
            self.df = pd.concat([self.df, self.data_setting()], ignore_index=True)
            #print(i)
        self.driver.quit()

# 사용 예시
kipris = Kipris()
kipris.scrape_data()

# 결과 데이터프레임을 CSV 파일로 저장
kipris.df.to_excel('patents_data.xlsx', index=False)
print("Data has been successfully written to 'patents_data.xlsx'")



pop_close() start
페이지 갯수 :  2
총 페이지 : ['2A8F4317C13AF846A76A85E7727CC601', '461540C35C384CA0505ADE6AE793CD82']
메인 페이지 코드 : 2A8F4317C13AF846A76A85E7727CC601
if end
pop_close() end

login() start

pop_close() start
페이지 갯수 :  2
총 페이지 : ['2A8F4317C13AF846A76A85E7727CC601', '810C7982D9400D7152283481F259CE14']
메인 페이지 코드 : 2A8F4317C13AF846A76A85E7727CC601
if end
pop_close() end
login() end
patent_move start
patent_move end

data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_setting start
data_setting end


data_set

ValueError: invalid literal for int() with base 10: '1,000'

In [4]:
!pip install openpyxl


   ---------------------------------------- 0.0/251.4 kB ? eta -:--:--
   ---------------------------------------- 251.4/251.4 kB 7.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:

kipris.df.to_excel('patents_data.xlsx', index=False)